# Optimizing memory usage 


In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

/home/lgrootde/Generative-re-tests/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
total_memory_bytes = torch.cuda.get_device_properties(0).total_memory

# Convert bytes to gigabytes
total_memory_gb = total_memory_bytes / (1024**3)

print(f"Total GPU Memory: {total_memory_gb:.2f} GB")

Total GPU Memory: 79.15 GB


In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Apr 11 16:48:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          Off | 00000001:00:00.0 Off |                    0 |
| N/A   31C    P0              62W / 300W |      9MiB / 81920MiB |      3%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [4]:
# name = "tiiuae/falcon-7b"
# model_8bit = AutoModelForCausalLM.from_pretrained(name, device_map="auto", load_in_8bit=True)
# model_norm = AutoModelForCausalLM.from_pretrained(name, device_map="auto", load_in_8bit=False)
# tokenizer = AutoTokenizer.from_pretrained(name)

In [5]:
# text = "The coolest thing about biology is"
# encoded_input = tokenizer(text, return_tensors='pt')
# output_sequences = model_8bit.generate(input_ids=encoded_input['input_ids'].cuda())
# print(f"8bit model output:{tokenizer.decode(output_sequences[0], skip_special_tokens=True)}")
# output_sequences = model_norm.generate(input_ids=encoded_input['input_ids'].cuda())
# print(f"normal model output:{tokenizer.decode(output_sequences[0], skip_special_tokens=True)}")

In [6]:
# mem_int8 = model_8bit.get_memory_footprint()
# mem_fp16 = model_norm.get_memory_footprint()
# print(f"8bit model memory footprint:{(mem_int8/(1024**3)):.2f} GB")
# print(f"normal model memory footprint:{(mem_fp16/(1024**3)):.2f} GB")
# print(f"difference: {(mem_fp16/mem_int8):.2f}")

### Multiple GPU:

according to [this guide](https://huggingface.co/docs/transformers/perf_train_gpu_many) there are multiple ways to train on multiple GPU's. the prefered way is using ZeRO - as it requires close to no modifications to the model. Because i have acces to double A10's i think i should be able to use the required NVLINK or NVSwitch. So i'll be exploring if that possible.

checking if nvlink or nvswitch is availible by using [this source](https://guide.ncloud-docs.com/docs/en/server-baremetal-a100-check-vpc#:~:text=Use%20the%20nvidia%2Dsmi%20topo,NVLink%20connection%20status%20between%20GPUs.&text=If%20NVLink%20is%20normally%20initialized,fabricmanager%20service%20is%20loaded%20normally.)

In [7]:
!systemctl status nvidia-fabricmanager.service

Unit nvidia-fabricmanager.service could not be found.


In [8]:
!systemctl status nv_peer_mem.service

Unit nv_peer_mem.service could not be found.


I'm guessing that the A10 GPU does not have nvlink / nvswitch